In [1]:
from PyPDF2 import PdfReader

# Define file paths
pdf_path = "r3064_buildatable.pdf"
txt_path = "r3064_buildatable.txt"



In [ ]:
# Extract text from PDF
pdf_reader = PdfReader(pdf_path)
text_content = "\n".join([page.extract_text() for page in pdf_reader.pages if page.extract_text()])

# Save extracted text to a .txt file
with open(txt_path, "w", encoding="utf-8") as txt_file:
    txt_file.write(text_content)

# Return the path of the extracted text file
txt_path

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load extracted text
with open(txt_path, "r", encoding="utf-8") as txt_file:
    document_text = txt_file.read()

# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,  # Maximum number of characters per chunk
    chunk_overlap=30  # Overlap to maintain context between chunks
)

# Split the text into chunks
text_chunks = text_splitter.split_text(document_text)

# Display a sample of the chunks
import pandas as pd
df_chunks = pd.DataFrame({"Chunk Number": range(1, len(text_chunks) + 1), "Text Chunk": text_chunks})
print(df_chunks.head())

   Chunk Number                                         Text Chunk
0             1  How to  \nBuild a Table\nAll the Basics Plus  ...
1             2  30"48"\n36"60"A good furniture maker is part a...
2             3  to furniture that is ugly, impractical or both...
3             4  this simplicity calls for a thorough knowledge...
4             5           Regardless of style, there are many mis-


In [3]:
import chromadb

# Initialize ChromaDB client
chroma_client = chromadb.PersistentClient(path="./chroma_db")  # Saves data persistently

# Create a collection
collection = chroma_client.get_or_create_collection(name="table_building_knowledge")

In [4]:
from sentence_transformers import SentenceTransformer

# Load embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Generate embeddings for each text chunk
embeddings = embedding_model.encode(text_chunks).tolist()  # Convert to list for ChromaDB

# Add chunks to ChromaDB
for i, (chunk, embedding) in enumerate(zip(text_chunks, embeddings)):
    collection.add(
        ids=[str(i)],  # Unique ID for each chunk
        embeddings=[embedding],  # Corresponding vector embedding
        metadatas=[{"text": chunk}]  # Store text as metadata
    )

print(f"✅ Successfully stored {len(text_chunks)} chunks in ChromaDB!")

/Users/avya/Desktop/chatbot/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Successfully stored 298 chunks in ChromaDB!


In [5]:
def retrieve_relevant_docs(query, top_k=10):
    """Retrieves the top K most relevant text chunks from ChromaDB."""
    query_embedding = embedding_model.encode([query]).tolist()  # Encode query
    results = collection.query(query_embeddings=query_embedding, n_results=top_k)
    
    retrieved_texts = [match["text"] for match in results["metadatas"][0]]
    return retrieved_texts

# Example Query
query = "What are the best materials for building a table?"
retrieved_docs = retrieve_relevant_docs(query)
print("\n".join(retrieved_docs))  # Display retrieved knowledge


How to  
Build a Table
All the Basics Plus  
One Beautiful Project Plan
and aiming to build a showpiece for your dining room, there are many aspects  of table design that you need to consider.
the table to be strong and attractive, and you don’t want wood movement of the top causing problems.
